In [1]:
from obstacles import DynamicObstacle
from geometries.geometry import Circle

obstacle = DynamicObstacle(
    id=2,
    geometry=Circle(radius=3),
    position=(0, 0),
    goal_position=(10, 10),
)